# EarthCube SPARQL Queries

## About

This is the start of learning a bit about the SPARQL to access the
EarthCube graph store.



## References



## Installs

In [1]:
!pip -q install mimesis
!pip -q install minio 
!pip -q install s3fs
!pip -q install SPARQLWrapper
!pip -q install boto3
!pip -q install 'fsspec>=0.3.3'
!pip -q install rdflib
!pip -q install rdflib-jsonld
!pip -q install PyLD==2.0.2

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
boto3 1.17.41 requires botocore<1.21.0,>=1.20.41, but you have botocore 1.19.52 which is incompatible.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
aiobotocore 1.2.2 requires botocore<1.19.53,>=1.19.52, but you have botocore 1.20.41 which is incompatible.


## Imports


In [4]:
import dask, boto3
import dask.dataframe as dd
import pandas as pd
import json
from string import Template

from SPARQLWrapper import SPARQLWrapper, JSON

sweet = "http://cor.esipfed.org/sparql"
dbsparql = "http://dbpedia.org/sparql"
ufokn = "http://graph.ufokn.org/blazegraph/namespace/ufokn-dev/sparql"
ecograph = "https://graph.geodex.org/blazegraph/namespace/nabu/sparql"

## Code inits

### Helper function(s)
The following block is a SPARQL to Pandas feature.  You may need to run it to load the function per standard notebook actions.

In [5]:
#@title
def get_sparql_dataframe(service, query):
    """
    Helper function to convert SPARQL results into a Pandas data frame.
    """
    sparql = SPARQLWrapper(service)
    sparql.setQuery(query)
    sparql.setReturnFormat(JSON)
    result = sparql.query()

    processed_results = json.load(result.response)
    cols = processed_results['head']['vars']

    out = []
    for row in processed_results['results']['bindings']:
        item = []
        for c in cols:
            item.append(row.get(c, {}).get('value'))
        out.append(item)

    return pd.DataFrame(out, columns=cols)

### Set up some Pandas Dataframe options

In [6]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)

### Set up the connection to the object store to access the graph objects from


Basic Query

```SPARQL
prefix schema: <http://schema.org/>
prefix sschema: <https://schema.org/>
SELECT DISTINCT ?s ?g  (MIN(?disurl) as ?url) (MAX(?score1) as ?score)   ?name ?description
WHERE {
  ?lit bds:search "${q}" .
  ?lit bds:matchAllTerms ${exact} .
  ?lit bds:relevance ?score1 .
  ?s ?p ?lit .

  VALUES (?dataset) { ( schema:Dataset ) ( sschema:Dataset ) }
  ?s a ?dataset .
  ?s schema:name|sschema:name ?name .

  graph ?g {
  ?s schema:description|sschema:description ?description .
  }

  OPTIONAL {
	?s schema:distribution|sschema:distribution ?dis .
	?dis schema:contentUrl |sschema:contentUrl  ?disurl .
  }

}
GROUP BY ?s ?g ?name ?description
ORDER BY DESC(?score1)
LIMIT ${n}
OFFSET ${o}
```

10 results for  for Carbon

In [7]:
query= """prefix schema: <http://schema.org/>
prefix sschema: <https://schema.org/>
SELECT DISTINCT ?s ?g  (MIN(?disurl) as ?url) (MAX(?score1) as ?score)   ?name ?description
WHERE {
  ?lit bds:search "Carbon" .
  ?lit bds:matchAllTerms true.
  ?lit bds:relevance ?score1 .
  ?s ?p ?lit .

  VALUES (?dataset) { ( schema:Dataset ) ( sschema:Dataset ) }
  ?s a ?dataset .
  ?s schema:name|sschema:name ?name .

  graph ?g {
  ?s schema:description|sschema:description ?description .
  }

  OPTIONAL {
	?s schema:distribution|sschema:distribution ?dis .
	?dis schema:contentUrl |sschema:contentUrl  ?disurl .
  }

}
GROUP BY ?s ?g ?name ?description
ORDER BY DESC(?score1)
LIMIT 10
OFFSET 0
       """
ecoresults = get_sparql_dataframe(ecograph, query)

print (ecoresults)


                                        s  \
0  https://www.bco-dmo.org/dataset/527438   
1  https://www.bco-dmo.org/dataset/540078   
2  https://www.bco-dmo.org/dataset/540138   
3  https://www.bco-dmo.org/dataset/527780   
4  https://www.bco-dmo.org/dataset/671199   
5  https://www.bco-dmo.org/dataset/828861   
6    https://www.bco-dmo.org/dataset/4041   
7  https://www.bco-dmo.org/dataset/664454   
8  https://www.bco-dmo.org/dataset/540098   
9  https://www.bco-dmo.org/dataset/540108   

                                                                    g  \
0  urn:gleaner:milled:bcodmo:ad43da2f953e0320f3e7d7305d09d7b9c47d5962   
1  urn:gleaner:milled:bcodmo:b794608720cb4e8638f135e26a7f53a73d8b9178   
2  urn:gleaner:milled:bcodmo:c571333fe646733978f124bc1a66f37905e0f832   
3  urn:gleaner:milled:bcodmo:c5b95020782360b1d0e3cd01270dfd1f6ea1ee48   
4  urn:gleaner:milled:bcodmo:ca8a2e54dafb686d5b1a9c3766629e7fb74666d1   
5  urn:gleaner:milled:bcodmo:ca991aadfc507fe671319966fe2368ecb632f

## Query with parameters

The queries in geocodes and geodex are parameterized.
Note while python templates usually use 
$identifier ${identifier} is equivalent to $identifier.

In [10]:

geocodesFullText = """
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
prefix schema: <http://schema.org/>
prefix sschema: <https://schema.org/>
SELECT distinct ?subj ?g ?resourceType ?name ?description  ?pubname (GROUP_CONCAT(DISTINCT ?placename; SEPARATOR=", ") AS ?placenames)
        (GROUP_CONCAT(DISTINCT ?kwu; SEPARATOR=", ") AS ?kw)
        ?datep  (GROUP_CONCAT(DISTINCT ?url; SEPARATOR=", ") AS ?disurl) (MAX(?score1) as ?score)

        WHERE {
            ?lit bds:search "${q}" .
            ?lit bds:matchAllTerms ${exact} .
            ?lit bds:relevance ?score1 .
            ?subj ?p ?lit .
            BIND (IF (exists {?subj a schema:Dataset .} ||exists{?subj a sschema:Dataset .} , "data", "tool") AS ?resourceType).
            filter( ?score1 > 0.04).
          graph ?g {
            Minus {?subj a sschema:ResearchProject } .
            Minus {?subj a schema:ResearchProject } .
            Minus {?subj a sschema:PropertyValue } .
            Minus {?subj a schema:PropertyValue } .
            Minus {?subj a schema:Person } .
            Minus {?subj a sschema:Person } .
             ?subj schema:name|sschema:name ?name .
                       ?subj schema:description|sschema:description ?description .
             }
            optional {?subj schema:distribution/schema:url|schema:subjectOf/schema:url ?url .}
            OPTIONAL {?subj schema:datePublished|sschema:datePublished ?date_p .}
            OPTIONAL {?subj schema:publisher/schema:name|sschema:publisher/sschema:name ?pub_name .}
            OPTIONAL {?subj schema:spatialCoverage/schema:name|sschema:spatialCoverage/sschema:name|sschema:sdPublisher ?place_name .}
            OPTIONAL {?subj schema:keywords|sschema:keywords ?kwu .}
            BIND ( IF ( BOUND(?date_p), ?date_p, "No datePublished") as ?datep ) .
            BIND ( IF ( BOUND(?pub_name), ?pub_name, "No Publisher") as ?pubname ) .
            BIND ( IF ( BOUND(?place_name), ?place_name, "No spatialCoverage") as ?placename ) .
        }
        GROUP BY ?subj ?pubname ?placenames ?kw ?datep ?disurl ?score ?name ?description  ?resourceType ?g
        ORDER BY DESC(?score)
LIMIT ${n}
OFFSET ${o}
"""
geocodesFullTextTemplate = Template(geocodesFullText)

In [11]:
q = 'Ridgecrest Earthquake'
exact = True # match all terms
o = 0 # start
n = 10 # limit number of results to 10

ridgecrest_query = geocodesFullTextTemplate.substitute(q=q,exact=exact,n=10,o=0)

ridgecrestresults = get_sparql_dataframe(ecograph, ridgecrest_query)
print(ridgecrestresults)

                                subj  \
0  https://doi.org/10.7283/HZN1-5910   
1  https://doi.org/10.7283/YJK0-B215   
2  https://doi.org/10.7283/N74Q-GA66   
3  https://doi.org/10.7283/5ASB-9V26   
4  https://doi.org/10.7283/5X1E-CR48   

                                                                    g  \
0  urn:gleaner:milled:unavco:5a0d55f5508ec2df891080bd878ad6ab88d867e3   
1  urn:gleaner:milled:unavco:032b4fbec6a37f19e09fc53b2c2bfba2b0e7630c   
2  urn:gleaner:milled:unavco:3c0ce6847d2045f44a0cf1edecbf7c2b8af29052   
3  urn:gleaner:milled:unavco:bf04e37ec94c5c295f1ada82c2fb70d5644b41fa   
4  urn:gleaner:milled:unavco:5cc1edda4117afe4f1e51d4db4e075677f2ee31c   

  resourceType name  \
0         data        
1         data        
2         data        
3         data        
4         data        

                                                                                    description  \
0  GPS/GNSS stations: Long-term continuous or semi-continuous occupations at multi

Selecting resources

```SPARQL
PREFIX schema:  <https://schema.org/>
PREFIX schemaold:  <http://schema.org/>
select DISTINCT ?rrs ?name ?curl ?landingPage
WHERE                    {
    graph <${g}>
      {
        {
          ?s schemaold:distribution|schema:distribution ?dist .
          ?dist  schemaold:encodingFormat|schema:encodingFormat ?type .
          ?dist schemaold:contentUrl|schema:contentUrl|schema:url|schemaold:url ?curl
        }
        UNION {
          VALUES (?dataset) { ( schema:Dataset ) ( schemaold:Dataset ) }
          ?s a ?dataset .
          ?s  schemaold:encodingFormat|schema:encodingFormat ?type .
          }
     }
     BIND (str(?type) as ?label)
     SERVICE <http://132.249.238.169:8080/fuseki/ecrr/query> {
      GRAPH <http://earthcube.org/gleaner-summoned>
       {
          ?rrs schema:supportingData ?df.
              ?df schema:encodingFormat  ?label ;
                  schema:position "input".
              ?rrs schema:name ?name.
               ?rrs schema:subjectOf/schema:url ?landingPage
       }
   }
}
````

In [56]:
getDownloadsQueryString = """PREFIX schema:  <https://schema.org/>
PREFIX schemaold:  <http://schema.org/>
select DISTINCT ?rrs ?name ?curl ?landingPage
WHERE                    {
    graph <${g}>
      {
        {
          ?s schemaold:distribution|schema:distribution ?dist .
          ?dist  schemaold:encodingFormat|schema:encodingFormat ?type .
          ?dist schemaold:contentUrl|schema:contentUrl|schema:url|schemaold:url ?curl
        }
        UNION {
          VALUES (?dataset) { ( schema:Dataset ) ( schemaold:Dataset ) }
          ?s a ?dataset .
          ?s  schemaold:encodingFormat|schema:encodingFormat ?type .
          }
     }
     BIND (str(?type) as ?label)
     SERVICE <http://132.249.238.169:8080/fuseki/ecrr/query> {
      GRAPH <http://earthcube.org/gleaner-summoned>
       {
          ?rrs schema:supportingData ?df.
              ?df schema:encodingFormat  ?label ;
                  schema:position "input".
              ?rrs schema:name ?name.
               ?rrs schema:subjectOf/schema:url ?landingPage
       }
   }
}
"""
getDownloadsTemplate = Template(getDownloadsQueryString)

# this will not work for now.
# returns boolean, no vars, so error in sqarql parse
hasDownloadsQueryString = """PREFIX sdo:  <https://schema.org/>

ASK
WHERE
{
  graph <urn:gleaner:milled:ocd:917529917c29eae1fcab0618f8f85f5587c771bb> {

   ?s  <https://schema.org/additionType> ?type .
	}
	  BIND (str(?type) as ?label)
   SERVICE <http://132.249.238.169:8080/fuseki/ecrr/query> {
	   GRAPH <http://earthcube.org/gleaner-summoned>
	   {
		 ?rrs sdo:supportingData/sdo:encodingFormat  ?label .
		 ?rrs sdo:name ?rrname.
		}
	}
 }
"""

hasDownloadsTemplate = Template(hasDownloadsQueryString)

In [57]:
# known working uri
o = 'urn:gleaner:milled:magic:c164daaca4ae58122d76ec48ecae1c1c45819fcf'

# hasDownloadsQuery = hasDownloadsTemplate.substitute(g=o)
# hasToolsResult = get_sparql_dataframe(ecograph, hasDownloadsQuery)

getDownloadsQuery = getDownloadsTemplate.substitute(g=o)
itemresults = get_sparql_dataframe(ecograph, getDownloadsQuery)

KeyError: 'vars'

batching a set of tools
Get a list of tools for a set of results.

hasTools not working. Returns a boolean. breaks some part of sparql stack.


In [49]:
# def hasTools( g):
#     hasDownloadsQuery = hasDownloadsTemplate.substitute(g=g)
#     return get_sparql_dataframe(ecograph, hasDownloadsQuery)
def addTools( g):
    getDownloadsQuery = getDownloadsTemplate.substitute(g=g)
    return get_sparql_dataframe(ecograph, getDownloadsQuery)

In [50]:
# has
# ridgecrestresults['hastools'] =ridgecrestresults['g'].apply(lambda g: hasTools( g))
ridgecrestresults['tools'] = ridgecrestresults['g'].apply(lambda g: addTools( g))
